In [49]:
# !pip install bayesian-optimization

In [50]:
# ML
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from bayes_opt import BayesianOptimization
import xgboost as xgb
import pickle

In [51]:
# df = pd.read_csv("Microbiota_composition.csv")
df = pd.read_csv("opti_df_AH.csv")
# df = df.drop("Unnamed: 0", axis=1)

In [52]:
# Features
X = df.drop(columns=["condition"], axis=1)

# Target
y = df["condition"]

# lambda to scale the target into numeric (CRC : 1, control : 0)
y = y.apply(lambda x: 1 if x == "CRC" else 0)

# Split df into train and test split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

# Boolean filter to keep numeric column
numeric_features = df.select_dtypes(include=['number']).columns
numeric_transformer = Pipeline(steps=[("scaler", StandardScaler())])

# Selection of the categorical feature to preprocess
# categorical_feature = X.select_dtypes(exclude=['number']).columns
# feature_transformer = Pipeline(steps=[("encoder", OneHotEncoder(drop="first"))])

# Use ColumnTransformer to make a preprocessor object that describes all the treatments to be done
preprocessor = ColumnTransformer(
    transformers=[
        ("num", numeric_transformer, numeric_features),
        # ("cat", feature_transformer, categorical_feature),
    ]
)

# Prétraitement des données (X_train, X_test) avec une normalisation
X_train = preprocessor.fit_transform(X_train)
X_test = preprocessor.transform(X_test)
# input_shape = X_train_scaled.shape[1]

# Définir la fonction d'évaluation
def evaluate_xgb(n_estimators, max_depth, learning_rate, gamma, min_child_weight):
    model = xgb.XGBClassifier(
        n_estimators=int(n_estimators),
        max_depth=int(max_depth),
        learning_rate=learning_rate,
        gamma=gamma,
        objective='binary:logistic',
        min_child_weight=min_child_weight,
        random_state=42
        )

    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    return accuracy

# Définir l'espace de recherche
pbounds = {
    'n_estimators': (10, 250),
    'max_depth': (3, 6),
    'learning_rate': (0.01, 0.1),
    'gamma': (0, 1),
    'min_child_weight': (0, 2)
    }

# Initialiser l'optimiseur bayésien
optimizer = BayesianOptimization(f=evaluate_xgb, pbounds=pbounds, random_state=42)

# Lancer l'optimisation
optimizer.maximize(init_points=10, n_iter=50)

# Récupérer les hyperparamètres optimaux
best_params = optimizer.max['params']

# Entraîner le modèle final avec les meilleurs hyperparamètres
best_model = xgb.XGBClassifier(n_estimators=int(best_params['n_estimators']),
                           max_depth=int(best_params['max_depth']),
                           learning_rate=best_params['learning_rate'],
                           gamma=best_params['gamma'],
                           min_child_weight=best_params['min_child_weight'])
best_model.fit(X_train, y_train)

# Enregistrez le modèle dans un fichier pickle
with open("xgboost_classifier_model.pkl", "wb") as file:
    pickle.dump(best_model, file)

# Charger le modèle depuis le fichier pickle
with open("xgboost_classifier_model.pkl", "rb") as file:
    loaded_model = pickle.load(file)

# Make predictions on the test set
# predictions = best_model.predict(X_test)

# Utilisez le modèle chargé pour faire des prédictions, par exemple
predictions = loaded_model.predict(X_test)

# Calculate accuracy
accuracy = accuracy_score(y_test, predictions)
print(f"Accuracy: {accuracy:.4f}")

|   iter    |  target   |   gamma   | learni... | max_depth | min_ch... | n_esti... |
-------------------------------------------------------------------------------------
| 1         | 0.7857    | 0.3745    | 0.09556   | 5.196     | 1.197     | 47.44     |
| 2         | 0.7662    | 0.156     | 0.01523   | 5.599     | 1.202     | 179.9     |
| 3         | 0.7662    | 0.02058   | 0.09729   | 5.497     | 0.4247    | 53.64     |
| 4         | 0.7532    | 0.1834    | 0.03738   | 4.574     | 0.8639    | 79.89     |
| 5         | 0.7597    | 0.6119    | 0.02255   | 3.876     | 0.7327    | 119.5     |
| 6         | 0.7143    | 0.7852    | 0.02797   | 4.543     | 1.185     | 21.15     |
| 7         | 0.7662    | 0.6075    | 0.02535   | 3.195     | 1.898     | 241.8     |
| 8         | 0.7468    | 0.8084    | 0.03742   | 3.293     | 1.368     | 115.6     |
| 9         | 0.7468    | 0.122     | 0.05457   | 3.103     | 1.819     | 72.11     |
| 10        | 0.7662    | 0.6625    | 0.03805   | 4.56